In [1]:
import copy

# 外皮情報LV2からLV3へのコンバート

## I. 関数

### 1. 主たる居室・その他の居室・非居室に接する外皮の部位の面積・長さ

#### 1) 面積

$$ A_{evlp,mr,i} = \frac{ A_{f,mr} }{ A_{f,total} } A_{evlp,i} $$

$$ A_{evlp,or,i} = \frac{ A_{f,or} }{ A_{f,total} } A_{evlp,i} $$

$$A_{evlp,nr,i} = \frac{ A_{f,total} - A_{f,mr} - A_{f,or} }{ A_{f,total}}   A_{evlp,i} $$

#### 2) 長さ

$$ l_{evlp,mr,i} = \frac{ A_{f,mr} }{ A_{f,total} } l_{evlp,i} $$

$$ l_{evlp,or,i} = \frac{ A_{f,or} }{ A_{f,total} } l_{evlp,i} $$

$$ l_{evlp,nr,i} = \frac{ A_{f,total} - A_{f,mr} - A_{f,or} }{ A_{f,total} } l_{evlp,i} $$

$A_{evlp,mr,i}$：主たる居室に接する外皮の部位（一般部位・開口部）$i$の面積, m<sup>2</sup>  
$A_{evlp,or,i}$：その他の居室に接する外皮の部位（一般部位・開口部）$i$の面積, m<sup>2</sup>  
$A_{evlp,nr,i}$：非居室に接する外皮の部位（一般部位・開口部）$i$の面積, m<sup>2</sup>  
$l_{evlp,mr,i}$：主たる居室に接する外皮の部位（熱橋）$i$の長さ, m  
$l_{evlp,or,i}$：その他の居室に接する外皮の部位（熱橋）$i$の長さ, m  
$l_{evlp,nr,i}$：非居室に接する外皮の部位（熱橋）$i$の長さ, m  
$A_{evlp,i}$ : 外皮の部位（一般部位・開口部）$i$の面積, m<sup>2</sup>  
$l_{evlp,i}$ : 外皮の部位（熱橋）$i$の面積, m  
$A_{f,total}$ : 床面積の合計, m<sup>2</sup>  
$A_{f,mr}$ : 主たる居室の床面積, m<sup>2</sup>  
$A_{f,or}$ : その他の居室の床面積, m<sup>2</sup>  

In [2]:
def get_separated_areas(a_evlp_i, a_f_mr, a_f_or, a_f_total):

    a_evlp_mr_i = a_f_mr / a_f_total * a_evlp_i
    a_evlp_or_i = a_f_or / a_f_total * a_evlp_i
    a_evlp_nr_i = (a_f_total - a_f_mr - a_f_or) / a_f_total * a_evlp_i
    
    return a_evlp_mr_i, a_evlp_or_i, a_evlp_nr_i

In [3]:
def get_separated_lengths(l_evlp_i, a_f_mr, a_f_or, a_f_total):

    l_evlp_mr_i = a_f_mr / a_f_total * l_evlp_i
    l_evlp_or_i = a_f_or / a_f_total * l_evlp_i
    l_evlp_nr_i = (a_f_total - a_f_mr - a_f_or) / a_f_total * l_evlp_i
    
    return l_evlp_mr_i, l_evlp_or_i, l_evlp_nr_i

$j=1$の時は主たる居室に接し、$j=2$の時はその他の居室に接し、$j=3$の時は非居室に接する

In [4]:
def get_space_types():
    
    return 'main_occupant_room', 'other_occupant_room', 'non_occupant_room'

In [5]:
def get_evlps_lv3_area(evlps_lv2, common_lv2, make_evlp_lv3):
    
    evlps_lv3 = []
    
    space_type_mr, space_type_or, space_type_nr = 'main_occupant_room', 'other_occupant_room', 'non_occupant_room'
    
    for evlp_lv2 in evlps_lv2:

        a_evlp_mr, a_evlp_or, a_evlp_nr = get_separated_areas(
            a_evlp_i=evlp_lv2['area'], 
            a_f_mr=common_lv2['main_occupant_room_floor_area'],
            a_f_or=common_lv2['other_occupant_room_floor_area'], 
            a_f_total=common_lv2['total_floor_area']
        )
        
        for space_type, a_evlp in zip((space_type_mr, space_type_or, space_type_nr), (a_evlp_mr, a_evlp_or, a_evlp_nr)):
            
            evlps_lv3.append(make_evlp_lv3(evlp_lv2, space_type, a_evlp))
            
    return evlps_lv3        

In [6]:
def get_evlps_lv3_length(evlps_lv2, common_lv2, make_evlp_lv3):
    
    evlps_lv3 = []
    
    space_type_mr, space_type_or, space_type_nr = 'main_occupant_room', 'other_occupant_room', 'non_occupant_room'
    
    for evlp_lv2 in evlps_lv2:

        l_evlp_mr, l_evlp_or, l_evlp_nr = get_separated_lengths(
            l_evlp_i=evlp_lv2['length'], 
            a_f_mr=common_lv2['main_occupant_room_floor_area'],
            a_f_or=common_lv2['other_occupant_room_floor_area'], 
            a_f_total=common_lv2['total_floor_area']
        )
        
        for space_type, l_evlp in zip((space_type_mr, space_type_or, space_type_nr), (l_evlp_mr, l_evlp_or, l_evlp_nr)):
            
            evlps_lv3.append(make_evlp_lv3(evlp_lv2, space_type, l_evlp))
            
    return evlps_lv3  

### 2. 外皮の部位の分類

#### 1) 一般部

In [7]:
def get_general_parts(general_parts, common):
    
    def make_general_part_lv3(evlp_lv2, space_type, a_evlp):
        return {
            'name'                  : evlp_lv2['name'] + '_' + space_type,
            'general_part_type'     : evlp_lv2['general_part_type'],
            'next_space'            : evlp_lv2['next_space'],
            'external_surface_type' : evlp_lv2['external_surface_type'],
            'direction'             : evlp_lv2['direction'],
            'area'                  : a_evlp,
            'space_type'            : space_type,
            'spec'                  : copy.deepcopy(evlp_lv2['spec'])
        }
    
    return get_evlps_lv3_area(general_parts, common, make_general_part_lv3)

#### Example

In [8]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'general_parts': [
            {
                'name'              : 'ceiling',
                'general_part_type' : 'ceiling',
                'next_space'        : 'outdoor', 
                'external_surface_type' : 'outdoor',
                'direction'         : 'top', 
                'area'              : 67.8,
                'spec'              : 'something',
            },
            {
                'name'              : 'floor',
                'general_part_type' : 'floor',
                'next_space'        : 'outdoor', 
                'external_surface_type' : 'outdoor',
                'direction'         : 'bottom', 
                'area'              : 67.8,
                'spec'              : 'something',
            },
            {
                'name'              : 'wall',
                'general_part_type' : 'wall',
                'next_space'        : 'outdoor', 
                'external_surface_type' : 'outdoor',
                'direction'         : 'n', 
                'area'              : 67.8,
                'spec'              : 'something',
            },
        ],
    }
} 

get_general_parts(d['envelope']['general_parts'], d['common'])

[{'name': 'ceiling_main_occupant_room',
  'general_part_type': 'ceiling',
  'next_space': 'outdoor',
  'external_surface_type': 'outdoor',
  'direction': 'top',
  'area': 16.95,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'ceiling_other_occupant_room',
  'general_part_type': 'ceiling',
  'next_space': 'outdoor',
  'external_surface_type': 'outdoor',
  'direction': 'top',
  'area': 33.9,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'ceiling_non_occupant_room',
  'general_part_type': 'ceiling',
  'next_space': 'outdoor',
  'external_surface_type': 'outdoor',
  'direction': 'top',
  'area': 16.95,
  'space_type': 'non_occupant_room',
  'spec': 'something'},
 {'name': 'floor_main_occupant_room',
  'general_part_type': 'floor',
  'next_space': 'outdoor',
  'external_surface_type': 'outdoor',
  'direction': 'bottom',
  'area': 16.95,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'floor_other_occupant_room',
  '

#### 2) 大部分がガラスで構成される窓等の開口部

In [9]:
def get_windows(windows, common):
    
    def make_window_lv3(window_lv2, space_type, a_evlp):
        return {
            'name'       : window_lv2['name'] + '_' + space_type,
            'next_space' : window_lv2['next_space'],
            'direction'  : window_lv2['direction'],
            'area'       : a_evlp ,
            'space_type' : space_type,
            'spec'       : copy.deepcopy(window_lv2['spec'])
        }

    return get_evlps_lv3_area(windows, common, make_window_lv3)

#### Example

In [10]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'windows': [
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.25,
                'spec'       : 'something',
            },
            {
                'name'       : 'window_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 3.17,
                'spec'       : 'something',
            },
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.25,
                'spec'       : 'something',
            },
            {
                'name'       : 'window_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 3.17,
                'spec'       : 'something',
            }
        ]
    }
}    

get_windows(d['envelope']['windows'], d['common'])

[{'name': 'window_sw_main_occupant_room',
  'next_space': 'outdoor',
  'direction': 'sw',
  'area': 7.5625,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'window_sw_other_occupant_room',
  'next_space': 'outdoor',
  'direction': 'sw',
  'area': 15.125,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'window_sw_non_occupant_room',
  'next_space': 'outdoor',
  'direction': 'sw',
  'area': 7.5625,
  'space_type': 'non_occupant_room',
  'spec': 'something'},
 {'name': 'window_nw_main_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 0.7925,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'window_nw_other_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 1.585,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'window_nw_non_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 0.7925,
  'space_type': 'non_occupant_room',
  'sp

#### 3) 大部分がガラスで構成されないドア等の開口部

In [11]:
def get_doors(doors, common):
    
    def make_door_lv3(door_lv2, space_type, a_evlp):
        return {
            'name'       : door_lv2['name'] + '_' + space_type,
            'next_space' : door_lv2['next_space'],
            'direction'  : door_lv2['direction'],
            'area'       : a_evlp,
            'space_type' : space_type,
            'spec'       : copy.deepcopy(door_lv2['spec'])
        }
    
    return get_evlps_lv3_area(doors, common, make_door_lv3)

#### Example

In [12]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'doors': [
            {
                'name'       : 'door_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 2.52,
                'spec'       : 'something'
            },
            {
                'name'       : 'doorNE',
                'next_space' : 'outdoor',
                'direction'  : 'ne',
                'area'       : 2.16,
                'spec'       : 'something'
            }
        ]
    }
}    

get_doors(d['envelope']['doors'], d['common'])

[{'name': 'door_nw_main_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 0.63,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'door_nw_other_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 1.26,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'door_nw_non_occupant_room',
  'next_space': 'outdoor',
  'direction': 'nw',
  'area': 0.63,
  'space_type': 'non_occupant_room',
  'spec': 'something'},
 {'name': 'doorNE_main_occupant_room',
  'next_space': 'outdoor',
  'direction': 'ne',
  'area': 0.54,
  'space_type': 'main_occupant_room',
  'spec': 'something'},
 {'name': 'doorNE_other_occupant_room',
  'next_space': 'outdoor',
  'direction': 'ne',
  'area': 1.08,
  'space_type': 'other_occupant_room',
  'spec': 'something'},
 {'name': 'doorNE_non_occupant_room',
  'next_space': 'outdoor',
  'direction': 'ne',
  'area': 0.54,
  'space_type': 'non_occupant_room',
  'spec': 'something'}]

#### 4) 熱橋

In [13]:
def get_heatbridges(heatbridges, common):

    def make_heatbridge_lv3(heatbridge_lv2, space_type, l_evlp):
        return {
            'name'       : heatbridge_lv2['name'] + '_' + space_type,
            'next_space' : heatbridge_lv2['next_space'],
            'direction'  : heatbridge_lv2['direction'],
            'space_type' : space_type,
            'length'     : l_evlp,
            'spec'       : copy.deepcopy(heatbridge_lv2['spec']),
        }

    return get_evlps_lv3_length(heatbridges, common, make_heatbridge_lv3)


#### Example

In [14]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'heatbridges': [
            {
                'name'       : 'heatbridge_ne',
                'next_space' : ['outdoor', 'outdoor'],
                'direction'  : ['n', 'e'],
                'length'     : 1.0,
                'spec'       : 'something',
            },
            {
                'name'       : 'heatbridge_nw',
                'next_space' : ['outdoor', 'outdoor'],
                'direction'  : ['n', 'w'],
                'length'     : 2.0,
                'spec'       : 'something',
            }
        ]
    }
}    

get_heatbridges(d['envelope']['heatbridges'], d['common'])

[{'name': 'heatbridge_ne_main_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'e'],
  'space_type': 'main_occupant_room',
  'length': 0.25,
  'spec': 'something'},
 {'name': 'heatbridge_ne_other_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'e'],
  'space_type': 'other_occupant_room',
  'length': 0.5,
  'spec': 'something'},
 {'name': 'heatbridge_ne_non_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'e'],
  'space_type': 'non_occupant_room',
  'length': 0.25,
  'spec': 'something'},
 {'name': 'heatbridge_nw_main_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'w'],
  'space_type': 'main_occupant_room',
  'length': 0.5,
  'spec': 'something'},
 {'name': 'heatbridge_nw_other_occupant_room',
  'next_space': ['outdoor', 'outdoor'],
  'direction': ['n', 'w'],
  'space_type': 'other_occupant_room',
  'length': 1.0,
  'spec': 'something'},
 {'name': 'heatbridge_nw_non_occupan

#### 5) 土間床等の外周部

In [15]:
def get_earthfloor_perimeters(earthfloor_perimeters):
    
    return [
        {
            'name'       : earthfloor_perimeter['name'],
            'next_space' : earthfloor_perimeter['next_space'],
            'direction'  : earthfloor_perimeter['direction'],
            'length'     : earthfloor_perimeter['length'],
            'space_type' : 'underfloor',
            'spec'       : copy.deepcopy(earthfloor_perimeter['spec']),
        }
        for earthfloor_perimeter in earthfloor_perimeters ]

example

In [16]:
d = {
    'earthfloor_perimeters': [
        {
            'name'       : 'earthfloor_perimeter_nw',
            'next_space' : 'outdoor',
            'direction'  : 'nw',
            'length'     : 2.43,
            'spec': 'something'
        },
        {
            'name'       : 'earthfloor_perimeter_ne',
            'next_space' : 'outdoor',
            'direction'  : 'ne',
            'length'     : 2.43,
            'spec': 'something'
        }
    ]
}    

get_earthfloor_perimeters(d['earthfloor_perimeters'])

[{'name': 'earthfloor_perimeter_nw',
  'next_space': 'outdoor',
  'direction': 'nw',
  'length': 2.43,
  'space_type': 'underfloor',
  'spec': 'something'},
 {'name': 'earthfloor_perimeter_ne',
  'next_space': 'outdoor',
  'direction': 'ne',
  'length': 2.43,
  'space_type': 'underfloor',
  'spec': 'something'}]

#### 6) 土間床等の中心部

In [17]:
def get_earthfloor_centers(earthfloor_centers):
    
    return [
        {
            'name'       : earghfloor_center['name'],
            'area'       : earghfloor_center['area'],
            'space_type' : 'underfloor'
        }
        for earghfloor_center in earthfloor_centers ]

example

In [18]:
d = {
    'earthfloor_centers': [
        { 'area': 5.0, 'name': 'other' },
        { 'area': 5.0, 'name': 'entrance' }
    ]
}    

get_earthfloor_centers(d['earthfloor_centers'])

[{'name': 'other', 'area': 5.0, 'space_type': 'underfloor'},
 {'name': 'entrance', 'area': 5.0, 'space_type': 'underfloor'}]

## II. 統合

In [19]:
def convert(d):
    
    common_lv2 = d['common']
    envelope_lv2 = d['envelope']
    
    common_lv3 = copy.deepcopy(common_lv2)
    envelope_lv3 = {}
    
    if 'general_parts' in envelope_lv2:
        envelope_lv3['general_parts'] = get_general_parts(envelope_lv2['general_parts'], common_lv2)
        
    if 'windows' in envelope_lv2:
        envelope_lv3['windows'] = get_windows(envelope_lv2['windows'], common_lv2)
        
    if 'doors' in envelope_lv2:
        envelope_lv3['doors'] = get_doors(envelope_lv2['doors'], common_lv2)
        
    if 'heatbridges' in envelope_lv2:
        envelope_lv3['heatbridges'] = get_heatbridges(envelope_lv2['heatbridges'], common_lv2)
        
    if 'earthfloor_perimeters' in envelope_lv2:
        envelope_lv3['earthfloor_perimeters'] = get_earthfloor_perimeters(envelope_lv2['earthfloor_perimeters'])
        
    if 'earthfloor_centers' in envelope_lv2:
        envelope_lv3['earthfloor_centers'] = get_earthfloor_centers(envelope_lv2['earthfloor_centers'])

    return {
        'common'   : common_lv3,
        'envelope' : envelope_lv3,
    }

example

In [20]:
d = {
    'common': {
        'region': 6,
        'main_occupant_room_floor_area': 30.0,
        'other_occupant_room_floor_area': 60.0,
        'total_floor_area': 120.0
    },
    'envelope' : {
        'general_parts': [
            {
                'name'       : 'ceiling',
                'general_part_type'  : 'ceiling',
                'next_space' : 'outdoor',
                'external_surface_type' : 'outdoor',
                'direction'  : 'top',
                'area'       : 67.8,
                'spec'       : 'something'
            },
            {
                'name'       : 'floor',
                'general_part_type'  : 'floor',
                'next_space' : 'outdoor',
                'external_surface_type' : 'outdoor',
                'direction'  : 'bottom',
                'area'       : 67.8,
                'spec'       : 'something'
            },
            {
                'name'       : 'wall',
                'general_part_type'  : 'wall',
                'next_space' : 'outdoor',
                'external_surface_type' : 'outdoor',
                'direction'  : 'n',
                'area'       : 67.8,
                'spec'       : 'something'
            },
            {
                'name'       : 'boundary_ceiling',
                'general_part_type'  : 'boundary_ceiling',
                'next_space' : 'air_conditioned',
                'external_surface_type' : 'outdoor',
                'direction'  : 'upward',
                'area'       : 67.8,
                'spec'       : 'something'
            },
        ],
        'windows': [
            {
                'name'       : 'window_sw',
                'next_space' : 'outdoor',
                'direction'  : 'sw',
                'area'       : 30.25,
                'spec'       : 'something'
            },
            {
                'name'       : 'window_nw', 
                'next_space' : 'outdoor', 
                'direction'  : 'nw', 
                'area'       : 3.17, 
                'spec'       : 'something'
            },
        ],
        'doors': [
            {
                'name'       : 'door_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'area'       : 2.52,
                'spec'       : 'something'
            },
        ],
        'heatbridges': [
            {
                'name'       : 'heatbridge_ne',
                'length'     : 1.00,
                'next_space' : ['outdoor', 'outdoor'],
                'direction'  : ['n', 'e'],
                'spec'       : 'something'
            },
        ],
        'earthfloor_perimeters': [
            {
                'name'       : 'earthfloor_perimeter_nw',
                'next_space' : 'outdoor',
                'direction'  : 'nw',
                'length'     : 2.43,
                'spec'       : 'something'
            },
        ],
        'earthfloor_centers': [
            { 'area': 5.0, 'name': 'other' },
            { 'area': 5.0, 'name': 'entrance' }
        ],
    }
}

convert(d)

{'common': {'region': 6,
  'main_occupant_room_floor_area': 30.0,
  'other_occupant_room_floor_area': 60.0,
  'total_floor_area': 120.0},
 'envelope': {'general_parts': [{'name': 'ceiling_main_occupant_room',
    'general_part_type': 'ceiling',
    'next_space': 'outdoor',
    'external_surface_type': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'main_occupant_room',
    'spec': 'something'},
   {'name': 'ceiling_other_occupant_room',
    'general_part_type': 'ceiling',
    'next_space': 'outdoor',
    'external_surface_type': 'outdoor',
    'direction': 'top',
    'area': 33.9,
    'space_type': 'other_occupant_room',
    'spec': 'something'},
   {'name': 'ceiling_non_occupant_room',
    'general_part_type': 'ceiling',
    'next_space': 'outdoor',
    'external_surface_type': 'outdoor',
    'direction': 'top',
    'area': 16.95,
    'space_type': 'non_occupant_room',
    'spec': 'something'},
   {'name': 'floor_main_occupant_room',
    'general_part_type': '